In [4]:
import time
import numpy as np


def load_matrix(filename, num_users, num_items):
    t0 = time.time()
    counts = np.zeros((num_users, num_items))
    total = 0.0
    num_zeros = num_users * num_items
    for i, line in enumerate(open(filename, 'r')):
        user, item, count = line.strip().split('\t')
        user = int(user)
        item = int(item)
        count = float(count)
        counts[user][item] = count
        total += count
        num_zeros -= 1
    alpha = num_zeros / total
    print ('alpha %.2f' % alpha)
    counts *= alpha
    t1 = time.time()
    print ('Finished loading matrix in %f seconds' % (t1 - t0))
    return counts

In [7]:
class LogisticMF():

    def __init__(self, counts, num_factors, reg_param=0.6, gamma=1.0,
                 iterations=30):
        self.counts = counts
        self.num_users = counts.shape[0] # 유저 수
        self.num_items = counts.shape[1] # 아이템 수
        self.num_factors = num_factors # latnet factor 수
        self.iterations = iterations # interation 수
        self.reg_param = reg_param # 정규화항
        self.gamma = gamma # learning rate

    # 학습
    # log 확률을 최대화시키는 x,y, 베타를 찾아야하므로 gradient ascent 진행
    def train_model(self):

        self.ones = np.ones((self.num_users, self.num_items)) # (user 수 * item 수) 의 값이 1인 행렬 만들기, 논문에서 R
        self.user_vectors = np.random.normal(size=(self.num_users,
                                                   self.num_factors)) # (유저 수 * latent factor) 의 유저 행렬 만들기, 논문에서 X
        self.item_vectors = np.random.normal(size=(self.num_items,
                                                   self.num_factors)) # (아이템 수 * latent factor)의 아이템 행렬 만들기, 논문에서 Y
        self.user_biases = np.random.normal(size=(self.num_users, 1)) # 유저 수만큼의 유저 편향 백터, 논문에서 베타u
        self.item_biases = np.random.normal(size=(self.num_items, 1)) # 아이템 수만큼의 아이템 편향 벡터, 논문에서 베타i

        user_vec_deriv_sum = np.zeros((self.num_users, self.num_factors)) # 유저 행렬의 미분 0으로 초기화
        item_vec_deriv_sum = np.zeros((self.num_items, self.num_factors)) # 아이템행렬의 미분 0으로 초기화
        user_bias_deriv_sum = np.zeros((self.num_users, 1)) # 유저 편향 벡터의 미분 0으로 초기화
        item_bias_deriv_sum = np.zeros((self.num_items, 1)) # 아이템 편향 벡터의 미분 0으로 초기화
        
        # iterations만큼 학습
        # 미분해서 추정값을 구할 때 Alternating Least Squares (ALS) 알고리즘을 사용한다.
        # -> 파라미터 추정 시 유저 파라미터(x, 베타u)와 아이템 파라미터(y, 베타i)를 번갈아가면서 추정한다.
        for i in range(self.iterations):
            t0 = time.time()
            # Fix items and solve for users
            # take step towards gradient of deriv of log likelihood
            # we take a step in positive direction because we are maximizing LL
            # 아이템 파라미터를 고정시키고 유저 파라미터를 추정한다.
            user_vec_deriv, user_bias_deriv = self.deriv(True) # 유저 파라미터의 미분 값을 구한다.
            # AdaGrad 수행
            user_vec_deriv_sum += np.square(user_vec_deriv) 
            user_bias_deriv_sum += np.square(user_bias_deriv)
            vec_step_size = self.gamma / np.sqrt(user_vec_deriv_sum)
            bias_step_size = self.gamma / np.sqrt(user_bias_deriv_sum)
            self.user_vectors += vec_step_size * user_vec_deriv # 유저 파라미터 값 갱신
            self.user_biases += bias_step_size * user_bias_deriv

            # Fix users and solve for items
            # take step towards gradient of deriv of log likelihood
            # we take a step in positive direction because we are maximizing LL
            # 유저 파라미터를 고정시키고 아이템 파라미터를 추정한다.
            item_vec_deriv, item_bias_deriv = self.deriv(False) # 아이템 파라미터의 미분 값을 구한다.
            # AdaGrad 수행
            item_vec_deriv_sum += np.square(item_vec_deriv)
            item_bias_deriv_sum += np.square(item_bias_deriv)
            vec_step_size = self.gamma / np.sqrt(item_vec_deriv_sum)
            bias_step_size = self.gamma / np.sqrt(item_bias_deriv_sum)
            self.item_vectors += vec_step_size * item_vec_deriv # 아이템 파라미터 값 갱신
            self.item_biases += bias_step_size * item_bias_deriv
            
            t1 = time.time()
            print('iteration %i finished in %f seconds' % (i + 1, t1 - t0))

    # 미분
    def deriv(self, user):
        if user:
            vec_deriv = np.dot(self.counts, self.item_vectors)
            bias_deriv = np.expand_dims(np.sum(self.counts, axis=1), 1)

        else:
            vec_deriv = np.dot(self.counts.T, self.user_vectors)
            bias_deriv = np.expand_dims(np.sum(self.counts, axis=0), 1)
        A = np.dot(self.user_vectors, self.item_vectors.T)
        A += self.user_biases
        A += self.item_biases.T
        A = np.exp(A)
        A /= (A + self.ones)
        A = (self.counts + self.ones) * A

        if user:
            vec_deriv -= np.dot(A, self.item_vectors)
            bias_deriv -= np.expand_dims(np.sum(A, axis=1), 1)
            # L2 regularization
            vec_deriv -= self.reg_param * self.user_vectors
        else:
            vec_deriv -= np.dot(A.T, self.user_vectors)
            bias_deriv -= np.expand_dims(np.sum(A, axis=0), 1)
            # L2 regularization
            vec_deriv -= self.reg_param * self.item_vectors
        return (vec_deriv, bias_deriv)

    # log 사후확률 (objective function)
    def log_likelihood(self):
        loglik = 0
        A = np.dot(self.user_vectors, self.item_vectors.T)
        A += self.user_biases
        A += self.item_biases.T
        B = A * self.counts
        loglik += np.sum(B)

        A = np.exp(A)
        A += self.ones

        A = np.log(A)
        A = (self.counts + self.ones) * A
        loglik -= np.sum(A)

        # L2 regularization
        loglik -= 0.5 * self.reg_param * np.sum(np.square(self.user_vectors))
        loglik -= 0.5 * self.reg_param * np.sum(np.square(self.item_vectors))
        return loglik

    # print vecotrs
    def print_vectors(self):
        user_vecs_file = open('logmf-user-vecs-%i' % self.num_factors, 'w')
        for i in range(self.num_users):
            vec = ' '.join(map(str, self.user_vectors[i]))
            line = '%i\t%s\n' % (i, vec)
            user_vecs_file.write(line)
        user_vecs_file.close()
        item_vecs_file = open('logmf-item-vecs-%i' % self.num_factors, 'w')
        for i in range(self.num_items):
            vec = ' '.join(map(str, self.item_vectors[i]))
            line = '%i\t%s\n' % (i, vec)
            item_vecs_file.write(line)
        item_vecs_file.close()
